In [2]:
import os
import pandas as pd
from datasets import load_dataset
import ast

## Load the data from Huggingface

In [ ]:
def convert_labels(example):
    example["Labels"] = [i for i in ast.literal_eval(example["Labels"])]
    return example

valueeval23 = load_dataset("webis/Touche23-ValueEval")
training_dataset = valueeval23["training"].map(convert_labels)
validation_dataset = valueeval23["validation"].map(convert_labels)
training_dataset['Labels']

## Load data from local files

Data PATH

In [3]:
data_dir = os.path.join('data')
arguments_training_filepath = os.path.join(data_dir, 'arguments-training.tsv')
arguments_validation_filepath = os.path.join(data_dir, 'arguments-validation.tsv')
arguments_validation_filepath_zhihu = os.path.join(data_dir, 'arguments-validation-zhihu.tsv')
arguments_test_filepath = os.path.join(data_dir, 'arguments-test.tsv')

labels_training_filepath = os.path.join(data_dir, 'labels-training.tsv')
labels_validation_filepath = os.path.join(data_dir, 'labels-validation.tsv')
labels_validation_filepath_zhihu = os.path.join(data_dir, 'labels-validation-zhihu.tsv')
labels_test_filepath = os.path.join(data_dir, 'labels-test.tsv')

Load the data

In [4]:
arguments_training = pd.read_csv(arguments_training_filepath, encoding='utf-8', sep='\t', header=0)
arguments_validation = pd.read_csv(arguments_validation_filepath, encoding='utf-8', sep='\t', header=0)
arguments_validation_zhihu = pd.read_csv(arguments_validation_filepath_zhihu, encoding='utf-8', sep='\t', header=0)
arguments_test = pd.read_csv(arguments_test_filepath, encoding='utf-8', sep='\t', header=0)


labels_training = pd.read_csv(labels_training_filepath, encoding='utf-8', sep='\t', header=0)
labels_validation = pd.read_csv(labels_validation_filepath, encoding='utf-8', sep='\t', header=0)
labels_validation_zhihu = pd.read_csv(labels_validation_filepath_zhihu, encoding='utf-8', sep='\t', header=0)
labels_test = pd.read_csv(labels_test_filepath, encoding='utf-8', sep='\t', header=0)


print(arguments_training.iloc[0], '\n', labels_training.iloc[0])

Argument ID                                               A01001
Conclusion                        Entrapment should be legalized
Stance                                               in favor of
Premise        if entrapment can serve to more easily capture...
Name: 0, dtype: object 
 Argument ID                   A01001
Self-direction: thought            0
Self-direction: action             0
Stimulation                        0
Hedonism                           0
Achievement                        0
Power: dominance                   0
Power: resources                   0
Face                               0
Security: personal                 0
Security: societal                 1
Tradition                          0
Conformity: rules                  0
Conformity: interpersonal          0
Humility                           0
Benevolence: caring                0
Benevolence: dependability         0
Universalism: concern              0
Universalism: nature               0
Universalism

## Some helper functions

In [5]:
def convert_binary_labels_to_string(df):
    label_names = df.columns[1:]
    string_labels = []

    for index, row in df.iterrows():
        binary_values = row.values[1:]
        string_labels.append([label_names[i] for i, value in enumerate(binary_values) if value == 1])

    df['String Labels'] = string_labels
    return df

def preprocess_arguments(df):
    prompt_format = "The premise: {}, is {}, the conclusion is {}. Which value category does it support?\n"
    preprocessed_arguments = []

    for _, row in df.iterrows():
        premise = row['Premise']
        stance = row['Stance']
        conclusion = row['Conclusion']
        prompt = prompt_format.format(premise, stance, conclusion)
        preprocessed_arguments.append(prompt)
        
    return preprocessed_arguments

def combine_columns(df_arguments, df_labels):
    """Combines the two `DataFrames` on column `Argument ID`"""
    return pd.merge(df_arguments, df_labels, on='Argument ID')

def generate_prompts(df):
    input_prompts = []
    output_prompts = []

    for _, row in df.iterrows():
        input_text = row['Text']
        labels = row['Labels']

        # Generate the input and output prompts
        input_prompt = "Text: {}".format(input_text)
        output_prompt = "Label: {}".format(labels)

        input_prompts.append(input_prompt)
        output_prompts.append(output_prompt)

    return input_prompts, output_prompts

In [6]:
processed_arguments = preprocess_arguments(arguments_training)
processed_arguments[0]

"The premise: if entrapment can serve to more easily capture wanted criminals, then why shouldn't it be legal?, is in favor of, the conclusion is Entrapment should be legalized. Which value category does it support?\n"

In [7]:
converted_labels = convert_binary_labels_to_string(labels_training)
print(converted_labels['String Labels'].iloc[7])

['Security: personal', 'Benevolence: caring', 'Universalism: concern']


In [8]:
argument = arguments_training.iloc[1]
print(argument)
sample = f"The premise: {argument['Premise']} is {argument['Stance']} conclusion: {argument['Conclusion']}. Which value category does it support? \n (A) societal (B) rules (C) interpersonal (D) none of the above"
sample

Argument ID                                               A01002
Conclusion                           We should ban human cloning
Stance                                               in favor of
Premise        we should ban human cloning as it will only ca...
Name: 1, dtype: object


'The premise: we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same. is in favor of conclusion: We should ban human cloning. Which value category does it support? \n (A) societal (B) rules (C) interpersonal (D) none of the above'

In [12]:
argument = processed_arguments[1]
argument = f"{argument} (A) societal (B) rules (C) interpersonal (D) none of the above"
argument

'The premise: we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same., is in favor of, the conclusion is We should ban human cloning. Which value category does it support?\n (A) societal (B) rules (C) interpersonal (D) none of the above'

In [10]:
from typing import Optional, List
from transformers import pipeline

from transformers import pipeline

generator = pipeline("text2text-generation", model="allenai/unifiedqa-t5-base")

2023-05-09 12:15:55.127038: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
generator(argument)

[{'generated_text': 'societal'}]